# NNDL课设报告
小组成员及对应工作量：苏柏侨(0.3)    李恒屹(0.4)    石基宽(0.3)

# 任务说明
服饰图像描述,训练一个模型,对输入的服饰图片,输出描述信息，我们实现的模型有以下三个实现：
- ARCTIC，一个典型的基于注意力的编解码模型
- 视觉Transformer (ViT) + Transformer解码器
- 网格/区域表示、Transformer编码器+Transformer解码器
  
同时也实现三种测评方法进行测评：
- BLEU (Bilingual Evaluation Understudy)
- SPICE (Semantic Propositional Image Caption Evaluation): 
- CIDEr-D (Consensus-based Image Description Evaluation)

以及实现了附加任务：
- 利用训练的服饰图像描述模型和多模态大语言模型，为真实背景的服饰图像数据集增加服饰描述和背景描述，构建全新的服饰图像描述数据集
  - 在新数据集上重新训练服饰图像描述模型

# 实验数据
数据集使用的是 DeepFashion-MultiModal (https://github.com/yumingj/DeepFashion-MultiModal), 仅用到image和textual descriptions ，数据集划分为10k+行数据的训练集和2k+行数据的测试集，`train_captions.json`和`test_captions.json`分别对应训练集和测试集的图片与描述信息的键值对应

In [8]:
img_path = f'../data/deepfashion-multimodal/images'
def cap_to_wvec(vocab,cap):#将文本描述转换成向量
    cap.replace(",","")
    cap.replace(".","")
    cap=cap.split()
    res=[]
    for word in cap:
        if word in vocab.keys():
            res.append(vocab[word])
        else: #不在字典的词
            res.append(vocab['<unk>'])
    return res
def wvec_to_cap(vocab,wvec):#将向量转换成文本描述
    res=[]
    for word in wvec:
        for key,value in vocab.items():
            if value==word and key not in ['<start>','<end>','<pad>','<unk>']:
                res.append(key)
    res=" ".join(res)
    return res
def wvec_to_capls(vocab,wvec):#将向量转换成文本描述
    res=[]
    for word in wvec:
        for key,value in vocab.items():
            if value==word and key not in ['<start>','<end>','<pad>','<unk>']:
                res.append(key)
    return res
class ImageTextDataset(Dataset):
    def __init__(self, dataset_path, vocab_path, split, captions_per_image=1, max_len=93, transform=None):

        self.split = split
        assert self.split in {'train', 'test'}
        self.cpi = captions_per_image
        self.max_len = max_len

        # 载入数据集
        with open(dataset_path, 'r') as f:
            self.data = json.load(f) #key是图片名字 value是描述
            self.data_img=list(self.data.keys())
        # 载入词典
        with open(vocab_path, 'r') as f:
            self.vocab = json.load(f)

        # PyTorch图像预处理流程
        self.transform = transform

        # Total number of datapoints
        self.dataset_size = len(self.data_img)

    def __getitem__(self, i):
        # 第i个文本描述对应第(i // captions_per_image)张图片
        img = Image.open(img_path+"/"+self.data_img[i]).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        c_vec=cap_to_wvec(self.vocab,self.data[self.data_img[i]])
        #加入起始和结束标志
        c_vec = [self.vocab['<start>']] + c_vec + [self.vocab['<end>']]
        caplen = len(c_vec)
        caption = torch.LongTensor(c_vec+ [self.vocab['<pad>']] * (self.max_len + 2 - caplen))
        
        return img, caption, caplen
        
    def __len__(self):
        return self.dataset_size
def mktrainval(data_dir, vocab_path, batch_size, workers=1):
    train_tx = transforms.Compose([
        transforms.Resize(256), # 重置图像分辨率
        transforms.RandomCrop(224), # 随机裁剪
        transforms.ToTensor(), # 转换成Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 标准化--三个参数为三个通道的均值和标准差
    ])
    val_tx = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    train_set = ImageTextDataset(os.path.join(data_dir, 'train_captions.json'), vocab_path, 'train',  transform=train_tx)
    test_set = ImageTextDataset(os.path.join(data_dir, 'test_captions.json'), vocab_path, 'test', transform=val_tx)

    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
    
    test_loader = torch.utils.data.DataLoader(
        test_set, batch_size=batch_size, shuffle=False, num_workers=workers, pin_memory=True, drop_last=False)

    return train_loader, test_loader    
train_loader,test_loader=mktrainval(data_dir='../data/deepfashion-multimodal',\
                                        vocab_path='../data/deepfashion-multimodal/vocab.json',\
                                        batch_size=3,workers=0) 
#workers=0 是因为ipynb不支持多线程

- 关于字典的处理
    我们采用了一个非常传统的方式，加载所有的训练集和测试集，进行词频统计，我们默认阈值为5 到达阈值的词将会加入到词典中

    之后我们额外添加了 < pad > < start > < end > < unk >四个词，分别代表填充词，句首标记，句尾标记，未知词
    
    最终写入到vocab.json文件中
- 关于数据集类的处理
我们使用Pytroch的Dataset来构建数据集类，在此之外封装了返回测试集和训练集的函数，可以进行自定义的批量预处理，我们在训练和推理过程中进行了如下的处理
    - resize 图像大小为256*256
    - 随机裁剪 为224*224
    - 转换为Torch Tensor
    - normalize 归一化为
        - mean=[0.485, 0.456, 0.406]
        - std=[0.229, 0.224, 0.225]

# 实验环境
- Python  3.9.16
- 主要依赖库
  - torch 
  - torchvision
  - nltk 
实际使用py库情况如下

项目开发中除数据集和模型外代码使用git进行版本控制，需要说明的是，由于课设由多个人多个设备下完成，而训练模型和数据集有所不同，我们维持同步的仅仅是Image2TextEvaluation这一项目，所以不同设备下由细微不同之处，如此测试，需要更换实际环境替换对应代码中的路径，直接进行测试可能会出现一些问题————这里提前声明这一点————文件路径大致如下：
```
├── Image2TextEvaluation  
│   ├── ARCTIC
│   │   ├── ARCTIC_dataloader.py
│   │   ├── ARCTIC_model.py
│   │   └── train.py
│   ├── Vit
│   │   ├── Vit.ipynb
│   │   ├── config.json
│   │   ├── generate.ipynb
│   ├── SwinTrans
│   │   ├── ...//同上
│   │   ├── evaluate.ipynb
│   │   └── gridSwinTrans.ipynb
│   │
│   ├── Tools
│   │   ├── test_blip.py
│   │   ├── ...
│   │
│   ├── new_dataset //Blip+多模态构建新数据集
│   │   ├── QianFan-agent.py 
│   │   ├── combined_input.json
│   │   ├── statement.txt
│   │   ├── merge_json.py
│   │   ├── ...
│   │
│   ├── evaluate.py
│   ├── README.md
│   └── 结题报告.ipynb
├── model
│   ├── best_arctic.ckpt
│   ├── last_arctic.ckpt
│   └── ...
├── data
│   ├── deepfashion-multimodal
│   │   ├── images
│   │   │   ├── 001.jpg
│   │   │   ├── ...
│   │   ├── train_captions.json
│   │   ├── test_captions.json
│   │   ├── vocab.json
│   │   └── ...

```


In [2]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence # 压紧填充序列
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from torchvision.models import ResNet101_Weights
from nltk.translate.bleu_score import corpus_bleu # BLEU评价指标
import numpy as np
import json
from torch.utils.data import Dataset
import os
from PIL import Image
from collections import Counter,defaultdict
from argparse import Namespace 


# 所用的方法或模型

## 评估方法

### BLEU (BiLingual Evaluation Understudy)
- BLUE是比较常用的评估指标之一，也是我们默认指标，需要注意的是，再调用计算BLEU值之前，要先将文本中人工添加的文本开始符、结束符和占位符去掉，其公式如下， 实际代码中我们借助nltk库进行实现
$$BLEU = \sum_{n=1}^k w_n \frac{ngram_{sys}(n)}{ngram_{ref}(n)}$$
其中：
  - n 是 n-gram 的阶数，取值范围为 1 到 4。
  - wn 是 n-gram 的权重，通常取均匀权重。
  - ngramsys(n) 是机器翻译结果中的 n-gram 数量。
  - ngramref(n) 是参考翻译中的 n-gram 数量。
  BLEU 的得分范围为 0 到 1。得分越高，表示机器翻译结果与参考翻译越相似。
  - 优点：容易计算
  - 缺点:
    - 没有考虑n-gram的顺序
    - 平等对待所有的n-gram
    - 衡量的是句子之间的流畅性而非语义相似度
### CIDEr-D (Consensus-based Image Description Evaluation)
- 是CIDEr的改进，对于动词原形和名词匹配成功的问题，CIDEr-D不再取词根
其用了一种折扣函数来降低长句子对评分的影响，增加了惩罚生成句子和参考句子的长度差别的权重，并且通过对n-gram计数的截断操作不再计算生成句子中出现次数超过参考句子的n-gram,
从而减少了重复单词对评分的影响，其实也是计算1到4 gram的结果的平均值，其公式如下
$$C I D E r - D _ { n } ( c _ { i } , S _ { i } ) = \frac { 1 0 } { m } \sum _ { j } e ^ { - \frac { -( i ( c _ { i } ) - l ( s _ { i j } ) ) ^ { 2 } } { 2 \sigma ^ { 2 } } } \times \frac { \min ( g ^ { n } ( c _ { i } ) , g ^ { n } ( s _ { i j } ) ) \cdot g ^ { n } ( s _ { i j } ) } {| | g ^ { n } ( c _ { i } ) | | | g ^ { n } ( s _ { i j } ) || } $$
- 优点：
  - CIDEr引入了TF-IDF为n-gram进行加权，这样就避免评价候选句子时因为一些常见却不够有信息量的n-gram打上高分
- 缺点：
  - CIDEr取词根的操作会让一些动词的原型和名词匹配成功
  - 高置信度的词重复出现的长句的CIDEr得分也很高
### SPICE (Semantic Propositional Image Caption Evaluation): 
- 是以名词为中心的度量，是以图的语义表示来编码图像描述中的对象、属性和关系
首先要将候选句子和参考句子集转化为场景图
然后比较候选句子和参考句子集中元组的precision、recall，最终计算出F1 score
公式如下
$$SPICE = \sum_{i=1}^m \frac{1}{|S_i|} \sum_{j=1}^n \frac{s_{ij}}{|R_i|}
$$
  - m 是图像描述的数量。
  - n 是图像描述中的对象、属性和关系的数量。
  - Si 是图像描述 i 中的对象、属性和关系。
  - Ri 是参考图像描述 i 中的对象、属性和关系。
  - sij 是图像描述 i 中的对象、属性和关系 j 与参考图像描述 i 中的对象、属性和关系 j 的相似度
- 优点：
  - 在语义而非n-gram层级度量
  - 每个句子映射到场景图后可以从中提取出模型关于某些关系或者属性的识别能力
- 缺点
  - 缺少n-gram来度量句子的流畅性
  - 度量的准确性受到场景图解析器的制约

使用代码如下，在evaluate的时候调用,接受cands, refs返回对应评估分数

In [12]:
def cider_d(reference_list, candidate_list, n=4):
    def count_ngrams(tokens, n):
        ngrams = []
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams.append(ngram)
        return ngrams

    def compute_cider_d(reference_list, candidate_list, n):
        cider_d_scores = []
        for refs, cand in zip(reference_list, candidate_list):
            cider_d_score = 0.0
            for i in range(1, n + 1):
                cand_ngrams = count_ngrams(cand, i)
                ref_ngrams_list = [count_ngrams(ref, i) for ref in refs]

                total_ref_ngrams = [ngram for ref_ngrams in ref_ngrams_list for ngram in ref_ngrams]

                count_cand = 0
                count_clip = 0

                for ngram in cand_ngrams:
                    count_cand += 1
                    if ngram in total_ref_ngrams:
                        count_clip += 1

                precision = count_clip / count_cand if count_cand > 0 else 0.0
                recall = count_clip / len(total_ref_ngrams) if len(total_ref_ngrams) > 0 else 0.0

                beta = 1.0
                f_score = (1 + beta**2) * precision * recall / (beta**2 * precision + recall) if precision + recall > 0 else 0.0

                cider_d_score += f_score

            cider_d_score /= n
            cider_d_scores.append(cider_d_score)

        return cider_d_scores

    reference_tokens_list = reference_list
    candidate_tokens_list = candidate_list

    scores = compute_cider_d(reference_tokens_list, candidate_tokens_list, n)

    return np.mean(scores)
def spice(reference_list, candidate_list, idf=None, beta=3):
    def count_ngrams(tokens, n):
        ngrams = []
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams.append(ngram)
        return ngrams

    def compute_spice_score(reference, candidate, idf, beta):
        reference_tokens = reference
        candidate_tokens = candidate

        reference_ngrams = [count_ngrams(reference_tokens, i) for i in range(1, beta + 1)]
        candidate_ngrams = [count_ngrams(candidate_tokens, i) for i in range(1, beta + 1)]

        precision_scores = []
        recall_scores = []

        for i in range(beta):
            common_ngrams = set(candidate_ngrams[i]) & set(reference_ngrams[i])

            precision = len(common_ngrams) / len(candidate_ngrams[i]) if len(candidate_ngrams[i]) > 0 else 0.0
            recall = len(common_ngrams) / len(reference_ngrams[i]) if len(reference_ngrams[i]) > 0 else 0.0

            precision_scores.append(precision)
            recall_scores.append(recall)

        precision_avg = np.mean(precision_scores)
        recall_avg = np.mean(recall_scores)

        spice_score = (precision_avg * recall_avg) / (precision_avg + recall_avg) if precision_avg + recall_avg > 0 else 0.0

        if idf:
            spice_score *= np.exp(np.sum([idf[token] for token in common_ngrams]) / len(candidate_tokens))

        return spice_score

    if idf is None:
        idf = {}

    spice_scores = []

    for reference, candidate in zip(reference_list, candidate_list):
        spice_score = compute_spice_score(reference, candidate, idf, beta)
        spice_scores.append(spice_score)

    return np.mean(spice_scores)
def get_BLEU_score(cands, refs): #获取BLEU分数
    multiple_refs = []
    for idx in range(len(refs)):
        multiple_refs.append(refs[(idx//1)*1 : (idx//1)*1+1])#每个候选文本对应cpi==1条参考文本
    bleu4 = corpus_bleu(multiple_refs, cands, weights=(0.25,0.25,0.25,0.25))
    return bleu4
def get_CIDER_D_score(cands, refs): #获得CIDER-D分数
    refs_ = [wvec_to_capls(model.vocab,ref) for ref in refs]
    cands_ = [wvec_to_capls(model.vocab,cand) for cand in cands]
    return cider_d(refs_, cands_)
def get_SPICE_score(cands, refs): #获得SPICE分数
    refs_ = [wvec_to_cap(model.vocab,ref) for ref in refs]
    cands_ = [wvec_to_cap(model.vocab,cand) for cand in cands]
    return spice(refs_, cands_)

## 模型定义
### ARCTIC，一个典型的基于注意力的编解码模型

模型架构: ARCTIC 是一个基于注意力的编码-解码模型，使用了图像编码器和注意力解码器。
#### 编码器部分
图像编码器使用了 ResNet-101 网络进行特征提取。

我们使用了预训练模型的权重，所以后续的训练中实际上编码器不参与训练过程，其参数是冻结的



我们并将其最后一个非全连接层作为网格表示提取层。
#### 解码器部分
解码器采用 GRU，利用注意力上下文向量和当前时刻的词嵌入来生成预测结果。该模型支持束搜索来生成更准确的描述。

解码器实质是一个rnn，其是有一层加性注意力机制，它接受查询（Q）和键值对（K，V），计算注意力分数，最后输出上下文向量。

- **加性注意力机制**：我们在AdditiveAttention 类上实现了加性注意力机制，用于在解码过程中关注图像中不同部分的信息。

- **使用GRU**: rnn具体使用的是 GRU ：是一种门控循环单元（gated recurrent unit）的缩写，它是一种用于处理序列数据的循环神经网络（RNN）
- 
    我个人认为选择GRU 而不是 LSTM 是因为相比LSTM，使用GRU能够达到相当的效果，并且相比之下更容易进行训练，能够很大程度上提高训练效率，可以很出现可观的效果。

- **forward过程：** 在单步forward过程中，我们将上下文向量和当前时刻的词表示拼接，然后和我们的rnn贴合在一起

    作为 GRU 的输入，进入全连接层对GRU 输出进行线性变换，得到单步forward；而实际生成句子的时候就是重复这个过程，选择概率最大的词作为下一个词，直到遇到结束符或者到达最大的长度

    当然，这种推理方法不是最好的，实质上这是一种 贪心算法
    我们知道，**贪心算法的缺点就是它无法保证全局最优**，我们要的是所有预测的的概率相乘最大。

- **使用束搜索**：所以我们还实现了另一种方法，即使用束搜索来生成更准确的描述。

    束搜索就是在每一步的时候，计算到这一步为止的预测y序列的概率最大的前k条，k叫集束宽。

    当然束搜索的缺点也很明显，推理所需时间实际上是倍增的，但是效果上是有增益的。



In [13]:
ARCTIC_config = Namespace(
        max_len = 93,
        captions_per_image = 1,
        batch_size = 32,
        image_code_dim = 2048,
        word_dim = 512,
        hidden_size = 512,
        attention_dim = 512,
        num_layers = 1,
        encoder_learning_rate = 0.0001,
        decoder_learning_rate = 0.0005,
        num_epochs = 10,
        grad_clip = 5.0,
        alpha_weight = 1.0,
        evaluate_step = 900, # 每隔多少步在验证集上测试一次
        checkpoint = None, # 如果不为None，则利用该变量路径的模型继续训练
        best_checkpoint = 'model/ARCTIC/best_ARCTIC.ckpt', # 验证集上表现最优的模型的路径
        last_checkpoint = 'model/ARCTIC/last_ARCTIC.ckpt', # 训练完成时的模型的路径
        beam_k = 5 #束搜索的束宽
    )
class ImageEncoder(nn.Module):
    def __init__(self, finetuned=True):
        super(ImageEncoder, self).__init__()
        model = torchvision.models.resnet101(weights=ResNet101_Weights.DEFAULT)
        # ResNet-101网格表示提取器
        self.grid_rep_extractor = nn.Sequential(*(list(model.children())[:-2])) #去掉最后两层 
        for param in self.grid_rep_extractor.parameters(): #冻结参数--不参与训练
            param.requires_grad = finetuned #是否微调
    def forward(self, images):
        out = self.grid_rep_extractor(images) 
        return out
class AdditiveAttention(nn.Module): #加性注意力
    def  __init__(self, query_dim, key_dim, attn_dim):
        """
            query_dim: 查询Q的维度
            key_dim: 键K的维度
            attn_dim: 注意力函数隐藏层表示的维度
        """
        
        super(AdditiveAttention, self).__init__()
        self.attn_w_1_q = nn.Linear(query_dim, attn_dim) #Q的线性变换
        self.attn_w_1_k = nn.Linear(key_dim, attn_dim) #K的线性变换
        self.attn_w_2 = nn.Linear(attn_dim, 1) #注意力函数隐藏层到输出层的线性变换
        self.tanh = nn.Tanh() #激活函数
        self.softmax = nn.Softmax(dim=1) #归一化函数

    def forward(self, query, key_value):
        """
        Q K V：Q和K算出相关性得分，作为V的权重，K=V
        参数：
            query: 查询 (batch_size, q_dim)
            key_value: 键和值，(batch_size, n_kv, kv_dim)
        """
        # （2）计算query和key的相关性，实现注意力评分函数
        # -> (batch_size, 1, attn_dim)
        queries = self.attn_w_1_q(query).unsqueeze(1) 
        # -> (batch_size, n_kv, attn_dim)
        keys = self.attn_w_1_k(key_value) #
        # -> (batch_size, n_kv)
        attn = self.attn_w_2(self.tanh(queries+keys)).squeeze(2)  #注意力评分函数
        # （3）归一化相关性分数
        # -> (batch_size, n_kv)
        attn = self.softmax(attn)  #归一化
        # （4）计算输出
        # (batch_size x 1 x n_kv)(batch_size x n_kv x kv_dim)
        # -> (batch_size, 1, kv_dim)
        output = torch.bmm(attn.unsqueeze(1), key_value).squeeze(1)
        return output, attn
class AttentionDecoder(nn.Module):
    def __init__(self, image_code_dim, vocab_size, word_dim, attention_dim, hidden_size, num_layers, dropout=0.5):
        super(AttentionDecoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, word_dim) #词嵌入 
        self.attention = AdditiveAttention(hidden_size, image_code_dim, attention_dim) #注意力机制
        self.init_state = nn.Linear(image_code_dim, num_layers*hidden_size) #初始化隐状态
        self.rnn = nn.GRU(word_dim + image_code_dim, hidden_size, num_layers) #GRU 门控循环
        self.dropout = nn.Dropout(p=dropout) #dropout
        self.fc = nn.Linear(hidden_size, vocab_size) #全连接层
        # RNN默认已初始化
        self.init_weights() #初始化权重
        
    def init_weights(self): #初始化权重
        self.embed.weight.data.uniform_(-0.1, 0.1) #词嵌入
        self.fc.bias.data.fill_(0) #全连接层
        self.fc.weight.data.uniform_(-0.1, 0.1) #全连接层
    
    def init_hidden_state(self, image_code, captions, cap_lens):
        """
        参数：
            image_code：图像编码器输出的图像表示 
                        (batch_size, image_code_dim, grid_height, grid_width)
        """
        # 将图像网格表示转换为序列表示形式 
        batch_size, image_code_dim = image_code.size(0), image_code.size(1)
        # -> (batch_size, grid_height, grid_width, image_code_dim) 
        image_code = image_code.permute(0, 2, 3, 1)  
        # -> (batch_size, grid_height * grid_width, image_code_dim)
        image_code = image_code.view(batch_size, -1, image_code_dim)
        # （1）按照caption的长短排序
        sorted_cap_lens, sorted_cap_indices = torch.sort(cap_lens, 0, True)
        captions = captions[sorted_cap_indices]
        image_code = image_code[sorted_cap_indices]
         #（2）初始化隐状态
        hidden_state = self.init_state(image_code.mean(axis=1))
        hidden_state = hidden_state.view(
                            batch_size, 
                            self.rnn.num_layers, 
                            self.rnn.hidden_size).permute(1, 0, 2)
        return image_code, captions, sorted_cap_lens, sorted_cap_indices, hidden_state

    def forward_step(self, image_code, curr_cap_embed, hidden_state):
        #（3.2）利用注意力机制获得上下文向量
        # query：hidden_state[-1]，即最后一个隐藏层输出 (batch_size, hidden_size)
        # context: (batch_size, hidden_size)
        context, alpha = self.attention(hidden_state[-1], image_code)
        #（3.3）以上下文向量和当前时刻词表示为输入，获得GRU输出
        x = torch.cat((context, curr_cap_embed), dim=-1).unsqueeze(0)
        # x: (1, real_batch_size, hidden_size+word_dim)
        # out: (1, real_batch_size, hidden_size)
        out, hidden_state = self.rnn(x, hidden_state)
        #（3.4）获取该时刻的预测结果
        # (real_batch_size, vocab_size)
        preds = self.fc(self.dropout(out.squeeze(0)))
        return preds, alpha, hidden_state
        
    def forward(self, image_code, captions, cap_lens):
        """
        参数：
            hidden_state: (num_layers, batch_size, hidden_size)
            image_code:  (batch_size, feature_channel, feature_size)
            captions: (batch_size, )
        """
        # （1）将图文数据按照文本的实际长度从长到短排序
        # （2）获得GRU的初始隐状态
        image_code, captions, sorted_cap_lens, sorted_cap_indices, hidden_state \
            = self.init_hidden_state(image_code, captions, cap_lens)
        batch_size = image_code.size(0)
        # 输入序列长度减1，因为最后一个时刻不需要预测下一个词
        lengths = sorted_cap_lens.cpu().numpy() - 1
        # 初始化变量：模型的预测结果和注意力分数
        predictions = torch.zeros(batch_size, lengths[0], self.fc.out_features).to(captions.device)
        alphas = torch.zeros(batch_size, lengths[0], image_code.shape[1]).to(captions.device)
        # 获取文本嵌入表示 cap_embeds: (batch_size, num_steps, word_dim)
        cap_embeds = self.embed(captions)
        # Teacher-Forcing模式
        for step in range(lengths[0]):
            #（3）解码
            #（3.1）模拟pack_padded_sequence函数的原理，获取该时刻的非<pad>输入
            real_batch_size = np.where(lengths>step)[0].shape[0]
            preds, alpha, hidden_state = self.forward_step(
                            image_code[:real_batch_size], 
                            cap_embeds[:real_batch_size, step, :],
                            hidden_state[:, :real_batch_size, :].contiguous())            
            # 记录结果
            predictions[:real_batch_size, step, :] = preds
            alphas[:real_batch_size, step, :] = alpha
        return predictions, alphas, captions, lengths, sorted_cap_indices
    
class ARCTIC(nn.Module): #模型主体部分
    def __init__(self, image_code_dim, vocab, word_dim, attention_dim, hidden_size, num_layers):
        super(ARCTIC, self).__init__()
        self.vocab = vocab
        self.encoder = ImageEncoder()
        self.decoder = AttentionDecoder(image_code_dim, len(vocab),
                                        word_dim, attention_dim, hidden_size, num_layers)
        print("test")
    def forward(self, images, captions, cap_lens):
        image_code = self.encoder(images)
        return self.decoder(image_code, captions, cap_lens)
    def generate_by_beamsearch(self, images, beam_k, max_len): # beam_k束搜索
        vocab_size = len(self.vocab)
        image_codes = self.encoder(images)
        texts = []
        device = images.device
        # 对每个图像样本执行束搜索
        for image_code in image_codes:
            # 将图像表示复制k份
            image_code = image_code.unsqueeze(0).repeat(beam_k,1,1,1)
            # 生成k个候选句子，初始时，仅包含开始符号<start>
            cur_sents = torch.full((beam_k, 1), self.vocab['<start>'], dtype=torch.long).to(device)
            cur_sent_embed = self.decoder.embed(cur_sents)[:,0,:]
            sent_lens = torch.LongTensor([1]*beam_k).to(device)
            # 获得GRU的初始隐状态
            image_code, cur_sent_embed, _, _, hidden_state = \
                self.decoder.init_hidden_state(image_code, cur_sent_embed, sent_lens)
            # 存储已生成完整的句子（以句子结束符<end>结尾的句子）
            end_sents = []
            # 存储已生成完整的句子的概率
            end_probs = []
            # 存储未完整生成的句子的概率
            probs = torch.zeros(beam_k, 1).to(device)
            k = beam_k
            while True:
                preds, _, hidden_state = self.decoder.forward_step(image_code[:k], cur_sent_embed, hidden_state.contiguous())
                # -> (k, vocab_size)
                preds = nn.functional.log_softmax(preds, dim=1)
                # 对每个候选句子采样概率值最大的前k个单词生成k个新的候选句子，并计算概率
                # -> (k, vocab_size)
                probs = probs.repeat(1,preds.size(1)) + preds
                if cur_sents.size(1) == 1:
                    # 第一步时，所有句子都只包含开始标识符，因此，仅利用其中一个句子计算topk
                    values, indices = probs[0].topk(k, 0, True, True)
                else:
                    # probs: (k, vocab_size) 是二维张量
                    # topk函数直接应用于二维张量会按照指定维度取最大值，这里需要在全局取最大值
                    # 因此，将probs转换为一维张量，再使用topk函数获取最大的k个值
                    values, indices = probs.view(-1).topk(k, 0, True, True)
                # 计算最大的k个值对应的句子索引和词索引
                sent_indices = torch.div(indices, vocab_size, rounding_mode='trunc') 
                word_indices = indices % vocab_size 
                # 将词拼接在前一轮的句子后，获得此轮的句子
                cur_sents = torch.cat([cur_sents[sent_indices], word_indices.unsqueeze(1)], dim=1)
                # 查找此轮生成句子结束符<end>的句子
                end_indices = [idx for idx, word in enumerate(word_indices) if word == self.vocab['<end>']]
                if len(end_indices) > 0:
                    end_probs.extend(values[end_indices])
                    end_sents.extend(cur_sents[end_indices].tolist())
                    # 如果所有的句子都包含结束符，则停止生成
                    k -= len(end_indices)
                    if k == 0:
                        break
                # 查找还需要继续生成词的句子
                cur_indices = [idx for idx, word in enumerate(word_indices) 
                               if word != self.vocab['<end>']]
                if len(cur_indices) > 0:
                    cur_sent_indices = sent_indices[cur_indices]
                    cur_word_indices = word_indices[cur_indices]
                    # 仅保留还需要继续生成的句子、句子概率、隐状态、词嵌入
                    cur_sents = cur_sents[cur_indices]
                    probs = values[cur_indices].view(-1,1)
                    hidden_state = hidden_state[:,cur_sent_indices,:]
                    cur_sent_embed = self.decoder.embed(
                        cur_word_indices.view(-1,1))[:,0,:]
                # 句子太长，停止生成
                if cur_sents.size(1) >= max_len:
                    break
            if len(end_sents) == 0:
                # 如果没有包含结束符的句子，则选取第一个句子作为生成句子
                gen_sent = cur_sents[0].tolist()
            else: 
                # 否则选取包含结束符的句子中概率最大的句子
                gen_sent = end_sents[end_probs.index(max(end_probs))]
            texts.append(gen_sent)
        return texts
mode_arctic="../best_arctic.ckpt"
model=torch.load(mode_arctic)["model"] #加载模型

### 视觉Transformer (ViT) + Transformer解码器




### ViT部分：
ViT是一种用于处理视觉数据的转换器架构。传统的卷积神经网络（CNN）在图像处理任务中表现出色，但ViT引入了自注意力机制，使得它可以处理可变大小的图像。ViT将输入图片分为多个patch（16x16），再将每个patch投影为固定长度的向量送入Transformer，后续encoder的操作和原始Transformer中完全相同。但是因为对图片分类，因此在输入序列中加入一个特殊的token，该token对应的输出即为最后的类别预测
一个ViT block可以分为以下几个步骤

- **patch embedding**
  例如输入图片大小为224x224，将图片分为固定大小的patch，patch大小为16x16，则每张图像会生成224x224/16x16=196个patch，即输入序列长度为196，每个patch维度16x16x3=768，线性投射层的维度为768xN (N=768)，因此输入通过线性投射层之后的维度依然为196x768，即一共有196个token，每个token的维度是768。这里还需要加上一个特殊字符cls，因此最终的维度是197x768。到目前为止，已经通过patch embedding将一个视觉问题转化为了一个seq2seq问题

- **positional encoding（standard learnable 1D position embeddings）**
  加入位置编码。加入位置编码信息之后，维度依然是197x768。位置编码的方式有**1-D位置编码**和**2-D 位置编码**，无论是哪种方式精度都很接近，甚至不适用位置编码性能损失也没有大到无法接受的地步。猜测因为ViT作用在image patch上，对网络来说这些patch之间的相对位置信息很容易理解，所以位置编码的方式与是否采用的影响都不大

- **LN/multi-head attention/LN**
  
  LN输出维度依然是197x768。将输入映射到q，k，v从而进行MHA操作，如果只有一个head，qkv的维度都是197x768，如果有12个头（768/12=64），则qkv的维度是197x64，一共有12组qkv，最后再将12组qkv的输出拼接起来，输出维度是197x768，然后在过一层LN，维度依然是197x768

- **MLP**
  
  将维度放大再缩小回去，197x768放大为197x3072，再缩小变为197x768一个block之后维度依然和输入相同，都是197x768，因此可以堆叠多个block。最后会将特殊字符cls对应的输出 作为encoder的最终输出 ，代表最终的image presentation（另一种做法是不加cls字符，对所有的tokens的输出做一个平均），后面接一个MLP进行图片分类

### Transformer解码器：
我们首先采用BERT来对提取的特征进行编码，再用Transformer解码器对编码内容进行处理后输出。要注意的是，这里我们仅采用了BERT的结构，并没有调用预训练模型。
BERT（Bidirectional Encoder Representations from Transformers）是一种自然语言处理中常用的预训练模型。我们将BERT用作图像特征的编码器。通过将ViT提取的图像特征输入BERT，模型能够学习图像中的上下文信息，捕捉图像内部的复杂关系。这使得模型能够更好地理解图像内容，从而更好地生成相关的图像描述。
- BERT的结构是标准transformer结构的encoder部分， 一个transformer的encoder单元由一个multi-head-Attention + Layer Normalization + feedforword + Layer Normalization 叠加产生，BERT的每一层由一个这样的encoder单元构成
- 这种transformer的结构可以使用上下文来预测mask的token，从而捕捉双向关系

之后，我们采用经典的Transformer解码器结构，Transformer解码器在NLP领域中被广泛用于生成序列数据，如文本。在这里，我们使用Transformer解码器来生成图像描述。解码器接收来自BERT编码器的图像特征作为输入，然后通过自注意力机制和前馈神经网络层逐步生成与图像内容相关的描述序列。这个描述序列最终形成图像的摘要或说明。

### 网格/区域表示、Transformer编码器+Transformer解码器

### 网格表示：
我们借鉴了 **SwinTransformer** 的结构来做网格特征提取。SwinTransformer是一个基于注意力机制的深度学习模型，专门设计用于图像处理任务。它采用了分层的注意力机制，允许模型有效地处理大尺寸的图像。在图像摘要生成任务中，SwinTransformer作为网格特征提取器，从输入图像中提取有用的特征。
- **Hierarchical Patch-based Attention**
  
  Swin Transformer引入了层次化的基于Patch的注意力机制。传统的Transformer模型是基于全连接的，对于大尺寸的图像，计算复杂度可能会非常高。为了解决这个问题，Swin Transformer将图像划分为一系列的非重叠图像块（patches），并在这些块上应用自注意力机制。这种分块处理允许Swin Transformer更好地扩展到大规模图像。
- **Shifted Windows**
  
  Swin Transformer采用了一种称为"shifted windows"的策略，通过改变注意力机制中的窗口偏移，提高了模型的局部感知能力。这对于捕捉图像中不同区域的特征非常有帮助，尤其是在需要考虑对象之间相对位置关系时。
- **Tokenization and Positional Embeddings**
  
  Swin Transformer将图像块转换为序列，每个序列元素对应一个图像块。为了使模型能够处理序列数据，需要引入类似自然语言处理中的tokenization和positional embeddings机制。这样，Swin Transformer可以对图像块进行有效的注意力计算，并理解它们之间的语义关系。

- **PatchMergin**
  
  类似于池化操作

- 借鉴了许多CNN中的trick，每经过一个stage后size就会缩小为原来的二分之一，channel扩大为两倍，与CNN相似
  - Patch Merging 模块将 尺寸为 H×WH×W 的 Patch 块首先进行拼接并在 channel 维度上进行 concatenate 构成了 H/2×W/2×4C的特征图，然后再进行 Layer Normalization 操作进行正则化，通过一个 Linear 层后形成了一个 H/2×W/2×2C得到特征图，完成了特征图的下采样过程。size 缩小为原来的 1/2，channel 扩大为原来的 2 倍。
- 分割成多个固定的窗口，每个窗口内的像素只能内部进行内积，虽然减少了计算开销，但是也因为这个操作，各个窗口无法进行信息交互，也即是感受野减小，难以整体获得信息，因此需要进行平移图像，改变分割方式，但这会使得计算量增加，因此重新滑动分割后的窗口，同时加入mask机制，防止不同位置的元素进行了自注意力的计算，最后再将信息数据平移回到原来的位置。通过 SW-MSA 机制完成了偏移窗口的像素点的 MSA 计算并实现了不同窗口间像素点的信息交流，从而间接扩大了网络的“感受野”，提高了信息的利用效率。


### Transformer编码器+Transformer解码器

同上，我们首先采用BERT来对提取的特征进行编码，再用Transformer解码器对编码内容进行处理后输出。要注意的是，这里我们仅采用了BERT的结构，并没有调用预训练模型。
BERT（Bidirectional Encoder Representations from Transformers）是一种自然语言处理中常用的预训练模型。我们将BERT用作图像特征的编码器。通过将SwinTransformer提取特征信息输入BERT，模型能够学习图像中的上下文信息，捕捉图像内部的复杂关系。这使得模型能够更好地理解图像内容，从而更好地生成相关的图像描述。
- BERT的结构是标准transformer结构的encoder部分， 一个transformer的encoder单元由一个multi-head-Attention + Layer Normalization + feedforword + Layer Normalization 叠加产生，BERT的每一层由一个这样的encoder单元构成
- 这种transformer的结构可以使用上下文来预测mask的token，从而捕捉双向关系

之后，我们采用经典的Transformer解码器结构，Transformer解码器在NLP领域中被广泛用于生成序列数据，如文本。在这里，我们使用Transformer解码器来生成图像描述。解码器接收来自BERT编码器的图像特征作为输入，然后通过自注意力机制和前馈神经网络层逐步生成与图像内容相关的描述序列。这个描述序列最终形成图像的摘要或说明。




In [ ]:
# ViT编码器+Transformer解码器
class Img2TxtModel(nn.Module):
    def __init__(self, vit_model_name, transformer_config, vocab_size):
        super(Img2TxtModel, self).__init__()
        # ViT模型作为编码器
        self.encoder = ViTModel.from_pretrained(vit_model_name)

        # Transformer解码器配置
        transformer_config = BertConfig(vocab_size=vocab_size, num_hidden_layers=1, is_decoder=True,  add_cross_attention=True)
        self.decoder = BertModel(transformer_config)

        # 预测每个词的线性层
        self.vocab_size = vocab_size
        self.fc = nn.Linear(transformer_config.hidden_size, vocab_size)
    
    def forward(self, input_ids, decoder_input_ids, decoder_attention_mask):
        # 通过ViT编码器获取图像特征
        encoder_outputs = self.encoder(pixel_values=input_ids).last_hidden_state

        # 将图像特征作为解码器的输入
        decoder_outputs = self.decoder(input_ids=decoder_input_ids, 
                                       attention_mask=decoder_attention_mask,
                                       encoder_hidden_states=encoder_outputs).last_hidden_state

        # 预测下一个词
        prediction_scores = self.fc(decoder_outputs)
        return prediction_scores

    def generate_text(self, input_ids, max_length=95, start_token_id=154):
        # 获取图像特征
        encoder_outputs = self.encoder(pixel_values=input_ids).last_hidden_state

        # 初始化解码器输入为<start>标记
        decoder_input_ids = torch.full((input_ids.size(0), 1), start_token_id).to(input_ids.device)
        
        # 存储所有时间步的logits
        all_logits = []

        for step in range(max_length):
            # 获取解码器输出
            decoder_outputs = self.decoder(
                input_ids=decoder_input_ids, 
                encoder_hidden_states=encoder_outputs
            ).last_hidden_state

            # 预测下一个词
            next_word_logits = self.fc(decoder_outputs[:, -1, :])
            all_logits.append(next_word_logits.unsqueeze(1))
            next_word_id = next_word_logits.argmax(dim=-1).unsqueeze(-1)
            
            # 将预测的词添加到解码器输入中
            decoder_input_ids = torch.cat([decoder_input_ids, next_word_id], dim=-1)
        
        return decoder_input_ids ,torch.cat(all_logits, dim=1)

In [ ]:
#SwinTransformerBlock ，SwinTransformer经过多个这样的block
class SwinTransformerBlock(nn.Module):
    def __init__(self, dim, num_heads, window_size=7, shift_size=0., mlp_ratio=4., qkv_bias=True, drop=0., attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super(SwinTransformerBlock, self).__init__()

        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio

        self.norm1 = norm_layer(dim)
        self.attn = self._create_attention_module(dim, num_heads, window_size, qkv_bias, attn_drop, drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = MLP(in_features=dim, hidden_features=mlp_hidden_dim, act=act_layer, drop=drop)

    def _create_attention_module(self, dim, num_heads, window_size, qkv_bias, attn_drop, drop):
        return WindowAttention(
            dim=dim, window_size=(window_size, window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop
        )

    def forward(self, x, attn_mask):
        x, Hp, Wp = self._process_input(x)
        shifted_x = self._shift_and_pad(x)
        x_windows = self._prepare_windows(shifted_x)
        attn_windows = self.attn(x_windows, mask=attn_mask)
        shifted_x = self._reverse_windows(attn_windows, Hp, Wp)
        x = self._restore_data(shifted_x, x.shape, Hp, Wp)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x

    def _process_input(self, x):
        H, W = self.H, self.W  # feature map
        B, L, C = x.shape
        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)
        return x, H, W

    def _shift_and_pad(self, x):
        if self.shift_size > 0.:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x
        return shifted_x

    def _prepare_windows(self, x):
        x_windows = window_partition(x, self.window_size)
        x_windows = x_windows.view(-1, self.window_size * self.window_size, x.shape[-1])
        return x_windows

    def _reverse_windows(self, attn_windows, Hp, Wp):
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, attn_windows.shape[-1])
        shifted_x = window_reverse(attn_windows, self.window_size, Hp, Wp)
        return shifted_x

    def _restore_data(self, shifted_x, original_shape, Hp, Wp):
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x_r = (self.window_size - original_shape[2] % self.window_size) % self.window_size
        x_d = (self.window_size - original_shape[1] % self.window_size) % self.window_size
        if x_r > 0 or x_d > 0:
            x = x[:, :original_shape[1], :original_shape[2], :].contiguous()
        x = x.view(original_shape[0], original_shape[1] * original_shape[2], original_shape[3])
        return x

In [ ]:
#特征提取器的总体结构
class SwinTransformerFeatureExtractor(nn.Module):
    def __init__(self, downsapmle_size=4, in_channels=3, embed_dim=96, depths=(2, 2, 6, 2), num_heads=(3, 6, 12, 24), window_size=7, mlp_ratio=4.,
                 qkv_bias=True, drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1, norm_layer=nn.LayerNorm, patch_norm=True, **kwargs):
        super(SwinTransformerFeatureExtractor, self).__init__()

        self.num_layers = len(depths)
        self.embed_dim = embed_dim
        self.patch_norm = patch_norm
        # stage4 输出的特征矩阵的Channel
        self.num_features = int(embed_dim * 2 ** (self.num_layers - 1))
        self.mlp_ratio = mlp_ratio

        self.patch_embed = patchEmbed(patch_size=downsapmle_size, in_channels=in_channels, embed_dim=embed_dim, norm_layer=norm_layer if self.patch_norm else None)
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]

        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers):
            layers = BasicLayer(dim=int(embed_dim * 2 ** i_layer),
                                depth=depths[i_layer],
                                num_heads=num_heads[i_layer],
                                window_size=window_size, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, drop=drop_rate, attn_drop=attn_drop_rate,
                                drop_path=dpr[sum(depths[:i_layer]):sum(depths[:i_layer + 1])], norm_layer=norm_layer, downsample=patchmerging if (i_layer < self.num_layers - 1) else None)
            self.layers.append(layers)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)

        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        # [B, L, C]
        x, H, W = self.patch_embed(x)
        x = self.pos_drop(x)
        hidden_states = []
        for layer in self.layers:
            x, H, W = layer(x, H, W)
            
            hidden_states.append(x.clone())

        return x, hidden_states


### blip + 多模态构建 新数据集

为了得到带有背景描述的数据集，我们利用训练的服饰图像描述模型和多模态大语言模型，为真实背景的服饰图像数据集增加服饰描述和背景描述，构建全新的服饰图像描述数据集。
我的使用的新数据集是选用 DeepFasion开源的12w数据集，仅使用图片，选用其中背景较丰富的5000张图像区间,具体范围可以看new_dataset/combined_input.json部分的起点和终点key。

由于没有条件调用GPT-4 V的API，我们没有直接使用多模态的大模型，而是采用img2txt再txt2txt的模式，先让图像描述模型（Blip）生成图像的文字描述，再用大语言模型（文心一言）接受这些文字描述，按照提示性的prompt生成更有深度的背景信息。

下面我封装了一个模块，用以快速调用Blip 进行批量处理，生成关于图片的简单文字表述，结果保存在new_dataset/res_new.json中，包括一个背景的粗略描述和一个整体的图像描述。


In [ ]:
class blip_model():
    def __init__(self) -> None:
        self.processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")
    def gen_res(self,img_path):
        raw_image = Image.open(img_path).convert('RGB')

        text = "a people in front of "
        input_1 = self.processor(raw_image, text, return_tensors="pt").to("cuda")
        out_1 = self.model.generate(**input_1,max_length=100)
        res_1=self.processor.decode(out_1[0], skip_special_tokens=True)

        input_2 = self.processor(raw_image, return_tensors="pt").to("cuda")
        out_2 = self.model.generate(**input_2,max_length=100)
        res_2=self.processor.decode(out_2[0], skip_special_tokens=True)
        return res_1+". "+res_2
def gen_json(img_path,n):
    model=blip_model()
    #img_path="D:/NNDL/data/deepfashion-multimodal/images"
    #获取该目录下所有文件，存入列表中
    imgs=os.listdir(img_path)
    res={}
    start=31000

    for img in range(start,len(imgs)):
        img_k=imgs[img]
        img_path_=img_path+"/"+img_k
        res[img_k]=model.gen_res(img_path_)
        if len(res)>=n:
            break
    #保存为json文件
    with open('res.json', 'w') as f:
        json.dump(res, f,indent=2)

### LLM生成更有深度的图像描述
将Blip生成的文字输入给文心一言，返回更有深度的背景图像描述。由于文心一言未能按照预期仅仅输出背景相关的描述，这里取描述的第一句话存于new_dataset/res_add.json中作为数据。

In [ ]:
import requests
import json
import os
import datetime

API_KEY = 'jIHlGsMYHp4j1MrMZGNmYbCL'  
SECRET_KEY = 'GzG1o4HC6G0qDVxPKcn9Zl4pv20j7CGA' #估计看到的时候已经过了有效期了所以无所谓了

headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    return str(requests.post(url, params=params).json().get("access_token"))

def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def QianFan(url, inputs):
    responses = {}  # 创建一个字典来存储输入和相应的响应

    for key, user_input in inputs.items():  # inputs是一个字典
        request = {
            "messages": [
            {
                "role": "user",
                "content": "I will give you a sentence, where the first part is a summary of the background and the second part is information about a person's outfit. Please focus on the background information from the first part and provide an overall background description. "
            },
            {
                "role": "assistant",
                "content": "Of course, please provide the sentence, and I will only output a sentence like: 'The background is'. "
            },
            {
                "role": "user",
                "content": "a people in front of a bed. a pair of jeans"
            },
            {
                "role": "assistant",
                "content": "The backgroud is a homely bedroom."
            },
            {
                "role": "user",
                "content": "a people in front of a mirror. a man in a blue shorts and a white shirt"
            },
            {
                "role": "assistant",
                "content": "The backgroud is a mirror."
            },
            ]
        }#添加一些前置词，以获得更贴合且标准化的回答。
        # 添加数据
        request["messages"].append({"role": "user", "content": f"Describe the setting of the following scene, focusing solely on the background without including any details about the person:{user_input}"})


        try:
            response = requests.request("POST", url, headers=headers, data=json.dumps(request))
            text = response.text
            data = json.loads(text)
            model_response = data['result']
            print("\n回答：\n", model_response, '\n')
            # 根据句号分割文本
            sentences = model_response.split(". ")

            # 获取第一个句子
            first_sentence = sentences[0] + "."
            responses[key] = first_sentence  # 将响应存储在字典中

        except Exception as e:
            print(f"QianFan 接口调用出错: {e}")

    # 保存或处理responses字典
    return responses

def main():
    if not os.path.exists('./Amadeus/history'):
        os.makedirs('./Amadeus/history')

    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/eb-instant?access_token=" + get_access_token()

    inputs = read_json('res_new.json') 
    responses = QianFan(url, inputs)

    # 可以选择保存responses字典
    with open('./Amadeus/history/responses.json', 'w', encoding='utf-8') as file:
        json.dump(responses, file, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    main()


### 生成基本的服饰描述
使用之前训练好的ViT模型，生成新数据集上的图像描述，结果储存在new_dataset/res.json中。
下面是用于调用之前训练的模型生成服装描述的脚本代码

In [ ]:
from torchvision import transforms
import json
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTFeatureExtractor, BertTokenizer ,ViTModel, BertModel, BertConfig
from collections import defaultdict, Counter
import numpy as np


dataset='deepfashion-multimodal'
img_path = f'data/{dataset}/img-001/img'
vocab_path = f'data/{dataset}/vocab.json'

def idx_to_word(idx, vocab):#将向量转化为文本描述
    reverse_vocab = {v: k for k, v in vocab.items()}
    return reverse_vocab.get(int(idx), '<unk>')

class CustomImageDataset(Dataset):
    def __init__(self, img_folder, transform=None):
        self.img_folder = img_folder
        self.img_names = [img for img in os.listdir(img_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]
        print(len(self.img_names))
        self.img_names = self.img_names[31000:36000]
        print(self.img_names[0])
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_folder, self.img_names[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, self.img_names[idx]
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

# 图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # 根据需要添加更多的转换
])

# 创建 Dataset 实例
dataset = CustomImageDataset(img_folder=img_path, transform=transform)

# 创建 DataLoader
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

with open(vocab_path, 'r') as f:
    vocab = json.load(f)

vocab_size = len(vocab)
vit_model_name = 'google/vit-base-patch16-224-in21k'
transformer_config = BertConfig()

model = Img2TxtModel(vit_model_name, transformer_config, vocab_size)
# 加载模型状态字典
checkpoint = torch.load('./model/best_model_epoch_10_batch_2700.pth')


# 将状态字典应用到模型实例中
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)

model.eval()  # 将模型设置为评估模式

generated_captions_dict = {}

with torch.no_grad():
    for images, name in data_loader:
        images = images.to(device)
        input_ids = images
        outputs,_ = model.generate_text(input_ids, max_length=95, start_token_id=vocab['<start>'])
        for i in range(outputs.shape[0]):
            gen_caption = [idx_to_word(idx, vocab) for idx in outputs[i]]
            if '<start>' in gen_caption:
                gen_caption = gen_caption[1:]  # 移除第一个元素 (<start>)
            if '<end>' in gen_caption:
                gen_caption = gen_caption[:gen_caption.index('<end>')]  # 移除 <end> 及其后面的元素

            caption_text = ' '.join(gen_caption)
            generated_captions_dict[name[0]] = caption_text
with open('res.json', 'w') as f:
    json.dump(generated_captions_dict, f, indent=2)

### 组合数据
使用脚本将服饰描述和背景描述拼接在一起，得到完整的数据。

In [ ]:
import json

# 读取JSON文件的函数
def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# 读取两个JSON文件
json1 = read_json('res.json')
json2 = read_json('res_add.json')

# 存储组合后的结果
combined_values = {}

# 遍历第一个JSON文件的键
for key in json1:
    if key in json2:
        # 将两个文件中相同键的值组合在一起
        combined_values[key] = json1[key] + json2[key]

# 保存组合后的结果到新的JSON文件
with open('combined_input.json', 'w', encoding='utf-8') as file:
    json.dump(combined_values, file, ensure_ascii=False, indent=2)

print("Combined JSON saved as combined_output.json")

### 再次训练模型
按照之前的流程重新在新数据集上训练一个模型，得到新的模型，这里不再放训练的代码。

# 实验结果
由于除了BLEU-4以外，其他指标的最大值其实和数据有关，所以为此我们先进行一个实验：得到如下指标的最大结果，然后将其作为一种相对的标准，结果如下：
BLEU-MAX:1.0|CIDEr-D-MAX:0.0043918896512309125|SPICE-MAX:0.18703616844830037，在评估过程中将会同时输出实际值和相对值（被压缩到0-1之间），

In [38]:
max_cider=0.0043918896512309125
max_spice=0.18703616844830037
def evaluate_(data_loader) :#用来测试剩下两个指标的最大值
    cands = []# 存储参考文本
    refs = []# 需要过滤的词
    filterd_words = set({model.vocab['<start>'], model.vocab['<end>'], model.vocab['<pad>']})
    for i, (imgs, caps, caplens) in enumerate(data_loader):
        cands.extend([filter_useless_words(cap, filterd_words) for cap in caps.tolist()])# 参考文本
        refs.extend([filter_useless_words(cap, filterd_words) for cap in caps.tolist()]) #候选文本
    bleu4_score = get_BLEU_score(cands, refs)
    cider_d_score = get_CIDER_D_score(cands, refs)
    spice_score= get_SPICE_score(cands, refs)
    print(f"BLEU-MAX:{bleu4_score}|CIDEr-D-MAX:{cider_d_score}|SPICE-MAX:{spice_score}")
    max_cider=cider_d_score
    max_spice=spice_score
evaluate_(test_loader)

BLEU-MAX:1.0|CIDEr-D-MAX:0.0043918896512309125|SPICE-MAX:0.18703616844830037


这里以 ARCTIC 的测评代码为例子

In [45]:
def filter_useless_words(sent, filterd_words):
    # 去除句子中不参与BLEU值计算的符号
    return [w for w in sent if w not in filterd_words]
cider_d_score=0
spice_score=0
def evaluate(data_loader, model, config):
    model.eval()
    # 存储候选文本
    cands = []
    # 存储参考文本
    refs = []
    # 需要过滤的词
    filterd_words = set({model.vocab['<start>'], model.vocab['<end>'], model.vocab['<pad>']})
    cpi = config.captions_per_image
    device = next(model.parameters()).device
    for i, (imgs, caps, caplens) in enumerate(data_loader):
        with torch.no_grad():
            # 通过束搜索，生成候选文本
            texts = model.generate_by_beamsearch(imgs.to(device), config.beam_k, config.max_len+2)
            # 候选文本
            cands.extend([filter_useless_words(text, filterd_words) for text in texts])
            # 参考文本
            refs.extend([filter_useless_words(cap, filterd_words) for cap in caps.tolist()])
    
    
    bleu4_score = get_BLEU_score(cands, refs)
    cider_d_score = get_CIDER_D_score(cands, refs)
    spice_score= get_SPICE_score(cands, refs)
    print(f"@@@实际值 BLEU:{bleu4_score}|CIDEr-D:{cider_d_score}|SPICE:{spice_score}")
    print(f"@@@相对值（0-1） BLEU:{bleu4_score}|CIDEr-D:{cider_d_score/max_cider}|SPICE:{spice_score/max_spice}")
    #model.train()
evaluate(test_loader, model, ARCTIC_config)

@@@实际值 BLEU:0.30466660274770024|CIDEr-D:0.0017447527516659645|SPICE:0.1336081641272871
@@@相对值（0-1） BLEU:0.30466660274770024|CIDEr-D:0.3972669830575009|SPICE:0.7143439968629298


同时我们以测试集的第一行数据来进行演示：以生成的描述文本和参考文本进行比较，直观评估ARCTIC的性能

In [24]:
def batch_eva(data_loader, model, config): #这里使用实验数据的第一个batch来进行演示
    model.eval()
    for i, (imgs, caps, caplens) in  enumerate(test_loader):
        cands = [] # 存储候选文本 
        refs = [] # 存储参考文本
        filterd_words = set({model.vocab['<start>'], model.vocab['<end>'], model.vocab['<pad>']}) #过滤词
        cpi = config.captions_per_image
        texts = model.generate_by_beamsearch(imgs.to("cuda"), config.beam_k, config.max_len+2)
        print("@生成数据：",texts)
        print("@生成的文本：",wvec_to_cap(model.vocab,texts[0])) #抽出一个batch的第一个文本
        print("@实际的文本：",wvec_to_cap(model.vocab,caps[0].tolist()))
        break
batch_eva(test_loader, model, ARCTIC_config)

@生成数据： [[154, 21, 47, 16, 31, 32, 39, 39, 52, 28, 10, 11, 12, 1, 39, 52, 16, 28, 7, 8, 9, 72, 13, 16, 84, 1, 49, 50, 28, 7, 8, 9, 10, 11, 12, 57, 16, 25, 59, 34, 35, 60, 155], [154, 21, 47, 16, 31, 32, 39, 39, 52, 28, 43, 12, 1, 39, 52, 16, 28, 7, 8, 9, 72, 13, 16, 84, 1, 49, 50, 28, 7, 8, 9, 43, 12, 57, 16, 25, 59, 34, 35, 60, 155], [154, 38, 40, 4, 5, 6, 7, 8, 9, 43, 12, 1, 13, 14, 15, 16, 81, 1, 18, 3, 16, 14, 5, 19, 1, 8, 16, 7, 9, 15, 4, 43, 12, 1, 26, 3, 16, 28, 7, 8, 9, 43, 12, 1, 26, 3, 16, 28, 7, 8, 9, 43, 12, 57, 16, 25, 59, 34, 35, 60, 155]]
@生成的文本： This person is wearing a tank tank top with solid color patterns. The tank top is with cotton fabric and its neckline is round. The pants are with cotton fabric and solid color patterns. There is an accessory on her wrist.
@实际的文本： This woman is wearing a tank tank shirt with graphic patterns and a three-point shorts. The tank shirt is with cotton fabric and its neckline is crew. The shorts are with cotton fabric and graphic patte

之后的两个模型的forward形式同上， 我们整理了最大值的统计情况,如下表格


| 模型名称 | BLEU-4 | CIDEr-D | SPICE |
|---------|------|------|------|
| ARCTIC   |  0.30466660274770024    |   (相对值：0.39726) 0.0017447527516659645  |   (相对值：0.71434)0.1336081641272871   |
| VIT   |    **0.3074786561430062**  |   (相对值：0.93790) 0.004119164250591897   |   (相对值：0.70628)0.1321016861247424   |
| SwinTrans   |   0.25770958160979623   |    (相对值：0.91071) 0.003999756354414652  |  (相对值：0.60681)0.11349623153205401    |

## 实验结果分析

根据实验结果，分析模型的优缺点
- BLEU-4 分析下
  - ARCTIC 模型 性能评价:
    ARCTIC 模型在 BLEU-4 上表现良好，得分为（0.30466660274770024）。
    语法和词汇的准确性得到了有效提升,而且经过实验贪心算法下的得分要低于束搜索算法下的得分，所以束搜索算法在生成文本时，能够生成更流畅的文本。
  - 视觉Transformer (ViT) + Transformer解码器 性能评价:
      ViT + Transformer 模型在 BLEU-4 上的得分为（0.3074786561430062）。模型在服饰描述任务中取得了良好的结果，生成文本在语和词汇方面表现出色。**在这个指标下 其语义流畅度最好**
  - 网格/区域表示、Transformer编码器+Transformer解码器 性能评价:  
      网格/区域表示 + Transformer 模型在 BLEU-4 上的得分为（0.25770958160979623）。
      模型在服饰描述任务中呈现出略低的性能，表现缺乏一定语句流畅度
- CIDEr-D 分析下
  - ARCTIC 模型 性能评价:
    ARCTIC 模型在 CIDEr-D 上取得了(相对值：0.39726) 0.0017447527516659645 
    CIDEr-D 分数显示模型在文本多样性和丰富性方面有一定的成功，但是不如其他模型。
  - 视觉Transformer (ViT) + Transformer解码器 性能评价:
    ViT + Transformer 模型在 CIDEr-D 上的得分为 (相对值：0.93790) 0.004119164250591897。
    模型在服饰描述任务中具有较高的文本多样性和丰富性，**是表现最好的模型**
  - 网格/区域表示、Transformer编码器+Transformer解码器 性能评价:
    网格/区域表示 + Transformer 模型在 CIDEr-D 上的得分为(相对值：0.91071) 0.003999756354414652。
    模型对服饰描述的多样性和丰富性取得了一定的成功。可以看出虽然流畅度不如其他模型，但是多样性还是不错的。
- SPICE 分析下
  - ARCTIC 模型 性能评价:
    ARCTIC 模型在 SPICE 上取得了 (相对值：0.71434)0.1336081641272871 。
    SPICE 分数反映了模型生成文本与图像内容相关性的程度，在语义层面上有一个很好的效果
  - 视觉Transformer (ViT) + Transformer解码器 性能评价:
    ViT + Transformer 模型在 SPICE 上的得分为 (相对值：0.70628)0.1321016861247424 。
    模型在服饰描述任务中表现出色，成功捕捉图像语义信息。
  - 网格/区域表示、Transformer编码器+Transformer解码器 性能评价:
    网格/区域表示 + Transformer 模型在 SPICE 上的得分为 (相对值：0.60681)0.11349623153205401。
    模型在 SPICE 上的表现显示其在描述图像内容方面的良好性能，但是效果相对较低

当然 ARCTIC、视觉Transformer (ViT) + Transformer解码器和网格/区域表示、Transformer编码器+Transformer解码器这三个模型有不同的优劣势

- ARCTIC 模型：
  - 优势：
    结合了注意力机制和编解码模型，有助于捕捉输入图像和生成描述之间的语义关系。
    注意力机制使得模型在生成描述时能够更加关注与服饰相关的区域，提高了描述的准确性。
  - 劣势：
    可能需要更多的计算资源和训练时间，因为结合了多个模型组件。
    在处理大规模数据集时，训练和推理速度可能较慢。
    推理过程中尝试了不同的生成方式，发现加入 beam search 策略效果最佳。


- 视觉 Transformer (ViT) + Transformer 解码器：
  - 优势：
    ViT 模型将输入图像转换为序列数据，直接应用 Transformer 解码器进行描述生成。
    Transformer 解码器在自然语言处理任务中表现出色，生成准确且流畅的描述。
  - 劣势：
    ViT 模型可能对输入图像的分辨率和细节要求较高，对于复杂的服饰图像可能需要更多的训练数据和计算资源。
    在处理长序列数据时，Transformer 解码器可能面临较长的训练和推理时间。


- 网格/区域表示、Transformer 编码器+Transformer 解码器：
  - 优势：
    网格/区域表示将图像划分为网格或区域，有助于捕捉局部特征。
    Transformer 编码器和解码器在处理序列数据时具有较强的建模能力，能够生成准确的描述。
  - 劣势：
    网格/区域表示可能需要额外的预处理步骤来划分图像，并可能导致信息损失。
    Transformer 编码器和解码器的训练和推理时间可能较长，特别是在处理大规模数据集时





# 新数据集分析
1."img_00035891.jpg": "a people in front of a red wall. a young man wearing a blue shirt and khaki pants" --->"img_00035891.jpg": "The scene takes place in front of a red wall."
2."img_00035892.jpg": "a people in front of a mirror. a woman wearing a t - shirt with a picture of a man"--->"img_00035892.jpg": "The setting is a mirror in a room."
3."img_00035899.jpg": "a people in front of a computer. a young boy sitting on a desk with a laptop"--->"img_00035899.jpg": "The setting is a room filled with a modern office desks and computers."
4."img_00035904.jpg": "a people in front of a flower. a little girl with a flower in her hand"--->"img_00035904.jpg": "The setting is a garden or park with flowers blooming."
在上述几个例子中，LLM接受blip生成的文字，确实可以输出更加有深度的背景信息，并且不会明显影响原本的信息，如1中的red wall。而且也能获得更深层的信息，如从mirror推出room，还有从laptop推出office，从flower推出花园
但是可惜的是，blip的功能比较弱，提取图片中央信息就已经不准确了，背景更是经常出错，如1中其实真实的背景是一个红色集装箱，而2中背景是窗帘和玻璃。

## 存在的问题 

- 模型训练部分
  - **资源不足**：不同模型对计算资源的需求不同，,作为学生的算力水平非常有限, 要训练出以该模型架构下的最优模型是一件困难的问题
  - **数据集限制**：以训练数据构建的词典其实是一个非常小的词典，其实限制了模型的上限，离开数据集，在真实数据下的推理可能还是会有不完备的地方
  - **图像裁剪**：数据集进行批量处理时的正方形裁剪可能会导致原始数据丢失部分信息，尤其是头部和脚步，还有白边导致的背景错误。
 
- 数据生成部分
  - **Blip性能不足**：我们使用的较小的模型，性能不足以提取所有背景，而且错误率出奇的高，导致后面的各项工作收到严重影响
    比如对于一个图像，经过blip生成的文本是：a people in front of a brick wall. a girl wearing a denim dress and white **shoes** ；但是原图没有鞋子的信息。在没有鞋子信息的情况下还是生成了鞋子信息
  - **LLM性能不足**：文心一言相比ChatGPT，理解能力还是较弱，而且除了第一句总结性的背景概括，后续便开始进行一些离奇的没有根据的幻想，比如下面这个例子中即使在我反复强调的情况下，LLM还是违背我对文本内容和长度的限制，生成如下长篇大论：
    The scene takes place in a room with a green mat on the floor. The background is a wall, which is painted in a neutral color and has no decorations or features. The room is dimly lit, with only a small amount of natural light coming from the window. There is a door leading out of the room, but it is closed. The man standing on the green mat is wearing a blue shorts and a white shirt, and he is facing the wall. He appears to be in a meditative or contemplative state, as he is standing still and not interacting with anything else in the room.
    但是，实际上我们需要的是相对较短的描述
   - **真实多模态能力的缺失**：由于现在的结构是先img2txt再txt2txt，对于LLM来说会损失很多图像信息，真正的多模态还是需要直接进行img、txt2txt，有一个badcase：img2txt模型将楼上的玻璃识别成了镜子，导致LLM的输出也出错，误认为其在一个房间里，而真实的图片是在一个街道上。
  这些因素导致数据集目前是一个不可用的状态。




# 总结

我们基本完成了作业要求，并额外进行了数据集的构建。

完成了数个模型的构建和训练。

利用训练的服饰图像描述模型和多模态大语言模型，为真实背景的服饰图像数据集增加服饰描述和背景描述，构建了一套可以生成经过了增强的全新数据集，并且在新数据集上重新训练服饰图像描述模型。

尽管在课设完成的过程中我们面临了很多挑战和问题，例如有限的算力水平，这可能影响了构建模型的性能和实验的严谨性，但是我们依然尽我们所能完成了实验任务。